# Workshop 1 - Water #

## Introduction ##

Water is the most common and important chemical solvent - it is not only central to life, it plays a critical role in colloid, interface and soft matter systems. Solvents such as water are typically handled in MD simulations via distinct force fields called _**solvent models**_, designed specifically to describe a solvent environment around a solute (e.g. micelle, polymer, protein) as accurately and efficiently as possible. Solvent models constitute a distinct group of MD force fields – they consist of parameters that are optimized to reproduce specific experimental properties of that particular liquid, such as density, enthalpies of vaporization and self-diffusion coefficients. 

As with any MD simulation, to simulate liquid water we must provide information about each water molecule’s initial position and velocity, as well as the forces that water molecules exert on each other. However, solvent water models involve additional parameters, such as whether each water molecule is flexible, or held fixed (and if so, the geometry of each water molecule), how the charge is distributed within each water molecule, and how each water molecule interacts with those around it via van der Waals interactions. Each of these parameters greatly affect how physically realistic the water model’s characteristics are. Each different water model will therefore have its own successes and weaknesses. 


### MD Water Models ###

A perfect model of liquid water would use quantum mechanics to exactly solve the system's Schrödinger equation, an as of yet insurmountable task, even on modern computers. Instead, assumptions are made to create water ‘models’ designed for different purposes to speed things up. Water models generally fix the geometry of individual molecules and replace their H and O atoms with point charges and describe van der Waal interactions via a simple Lennard-Jones potential. In other words, the energy of water is described by, 

$$E_{ab} = \sum_{i}^{i \in \text{mol. A}}\sum_{j}^{j \in \text{mol. B}}\frac{k_C q_i q_j}{r_ij} + \frac{A}{r_{OO}^{12}} - \frac{B}{r_{OO}^{6}}$$

Notice that this expression does not depend on the distance between H and O atoms in the **same** water molecule. This means that the geometry in each water molecule is fixed. The total interaction between pairs of water molecules using this model therefore arises from 3 sites, and is just the sum of their electrostatic interaction (the first term), and their van der Waals interactions (the 2nd and 3rd terms). The latter is assumed to arise only from the oxygen atom. 

Simulating liquid water with 3-site water models therefore only requires **4 parameters** ($q_{O}$, $q_{H}$, $A$ and $B$) - this makes MD calculations extremely efficient. However, since there are only 4 parameters, water models are notoriously inflexible. For instance, values of $q_{O}$, $q_{H}$, $A$ and $B$ that give an accurate prediction of one property, such as water density, may give an inaccurate prediction of the heat of vaporisation. In the Table below, TIP3P and SPC/E are both examples of 3-site water models which we will use in this workshop.

One of the most popular attempts to solve this problem is to introduce "fake" atoms into the water molecule structure. These fictitious **dummy atoms** ("M") atoms have no mass, but a non-zero charge. They are generally placed near the oxygen atom, effectively changing the water molecules dipole moment - and therefore the  electrostatic and van der Waals interactions between pairs of water molecules. Effectively the presence of the dummy atom in this water model mimics, in a crude way, the presence of the oxygen lone electron pairs. Such "4-site" water models rely on **5 parameters** ($q_{O}$, $q_{H}$,$q_{M}$, $A$ and $B$), so have an increased degree of flexibility and can predict some properties of liquid water more accurately. However this comes at the cost of (1) having to determine what the charge on the dummy atom M is, and where it should reside in the water molecule (see **r(OM)** in the table below). More importantly, the inclusion of an additional atom in the force field means that MD simulations will be slightly less efficient. In the Table below, TIP4P-Ew, TIP4P/Ice, TIP4P/2005 and OPC are all examples of 4-site water models, each designed to predict some specific property of water (for instance, the parameters of the TIP4P/Ice model were specifically designed for simulating different phases of solid water). In this workshop we will examine the performance of TIP4P-Ew and OPC.

Yet more advanced water models have also been developed, such as the 5- and 6-site water models. An example of the 5-site water model is shown below - instead of a single dummy atom M, 5-site water models incorporate a pair of dummy atoms ("L") that mimic more accurately the position of the oxygen lone electron pairs. 6-site models incorporate these lone pair dummy atoms and the 4-site dyummy atom M together. The incorporation of multiple dummy atoms generally improves the prediction of these models, but with a significant increase in model complexity and simulation  efficiency.





<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/Water_models.svg/1024px-Water_models.svg.png" align="center"/>

**Table 1: Examples of 3- & 4-Site Water Models**
| Model | $q_{O}$ (e) | $q_{H}$ (e) | $q_{M}$ (e) | r(OH) ($\mathring A$) | $\theta_{\text{HOH}}$ (deg.) |  r(OM) ($\mathring A$)  | A ($10^{3}$kcal$\mathring A^{12}$mol$^{-1}$)  | B  ($10^{3}$kcal$\mathring A^{12}$mol$^{-1}$) |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| TIP3P | -0.834 | +0.417| -  | 0.9572 | 104.52 | - | 582.0 | 595.0 | 
| SPC/E | -0.8476 | +0.4238 | - | 1.0 | 109.47 | - | 629.4 | 625.5 |
| TIP4P-Ew | - | +0.52422 | −1.04844 | 0.9572 | 104.52 | 0.125 | 656.1  | 653.5  |
| TIP4P/Ice | - | +0.5897 | −1.1794 | 0.9572 | 104.52 | 0.1577 | 857.9  | 850.5 |
| TIP4P/2005 | - | +0.5564 | −1.1128 | 0.9572 | 104.52 | 0.1577 | 731.3  | 736.0 |
| OPC | | +0.6791 |  −1.3582 | 0.8724 | 103.6 | 0.1546 | 865.1 | 858.1 |
| **Exp. water (gas-phase)**  |- |- | - | **0.9572** | **104.52** | - | - |

### Workshop Aims ###

In this workshop, we examine how well different water models reproduce structural and dynamic properties of water under various physical conditions.

The workshop addresses three questions:
 
1. How reliable are different water models in reproducing experimental data for liquid water dynamics and structure at 298 K and 1 atm?
1. How reliable are different water models in reproducing temperature-dependent properties of liquid water?
1. How well does each water model reproduce the structure of water at a given temperature and pressure? Specifically, can the model correctly predict the phase of water at 250 K and 100 atm?


## Simulating Bulk Water ##

### TIP3P Water Simulation ###

#### Setting Things Up  ###

First things first - as we saw in the previous introductory exercise, several modules need to be first loaded into our Jupyter notebook so that we can run our MD simulation. Let's take care of this now:

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import re
import time
import numpy as np

Now let's define the physical parameters of our simulation. You can use the panels below to change them, however for now it is ok to leave them at their present values. 

In [9]:
import ipywidgets as widgets
from IPython.display import display

#SIMULATION TEMPERATURE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=298.0,  # default value 
    description='Simulation Temperature (K):',
    step=1.0,  # step size
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simTemperature
    simTemperature = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simTemperature = temp_text.value*kelvin

#BOXSIZE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=48.0,  # default value 
    description='Box Size (Angstrom):',
    step=1.0,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global boxsize
    boxsize = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
boxsize = temp_text.value

#TIMESTEP
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1000.0,  # default value 
    description='Time step (fs):',
    step=100.0,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simTimestep
    simTimestep = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simTimestep = temp_text.value*femtoseconds

#PRESSURE
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1.0,  # default value 
    description='Simulation pressure (atm):',
    step=0.10,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simPressure
    simPressure = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simPressure = temp_text.value*atmospheres

#STEPS
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=10000,  # default value 
    description='# MD steps:',
    step=100,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global simNumSteps
    simNumSteps = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
simNumSteps = temp_text.value

#WATERS
# create a float text widget to input variable
temp_text = widgets.FloatText(
    value=1000,  # default value 
    description='# MD steps:',
    step=10,  # step size 
)
# adjust the layout of the widget
temp_text.layout.width = 'auto'
temp_text.style.description_width = 'initial'
# define a function to update the variable
def update_temp_text(change):
    global water
    simNumSteps = change.new
# register the update function with the widget
temp_text.observe(update_temp_text, 'value')
# display the widget
display(temp_text)
# access the selected temperature value
water = temp_text.value

FloatText(value=298.0, description='Simulation Temperature (K):', step=1.0)

Once you're happy with your simulation parameters, let's make sure their stored correctly by executing the next cell:

In [ ]:
print(f"Simulation size: {water} waters")
print(f"Box size: {boxsize} Ang.")
print(f"Simulation temperature: {simTemperature} K")
print(f"Simulation pressure: {simPressure} atm")
print(f"Simulation time step: {simTimestep} fs.")
print(f"# MD steps: {simNumSteps} ; Total simulation length: {simNumSteps * simTimestep} fs = {simNumSteps*simTimestep / 1000 / 1000} ns.")

In [ ]:
#System parameters
#water=1000
#boxsize=48.0
#packing_tolerance = 2.0
#starting_pdb_file = "system.pdb"
#starting_box_size = 48.0
#simTemperature = 300*kelvin
#simTimestep = 1*femtoseconds
#simPressure = 1*atmospheres
#simNumSteps = 1000
#pdbFile = 'system.pdb'

#### Generating the Unit Cell ####

With the parameters for our simulation now defined, we need to generate the actual "box" of water molecules that we will use for the simulation. We will use an external program call ```packmol``` to do this. ```packmol``` is widely used for generating condensed phase systems in biology, materials science and colloid & interface science. Here we are only generating a water box, but it is capable of [much more complicated stuff](https://m3g.github.io/packmol/examples.shtml) as well. 

To start with, we will prepare the input file, ```packmol.in```, that ```packmol``` will go looking for. This input file tells ```packmol``` to build a box of waters according to the parameters that you just set above:

In [ ]:
with open("packmol.in", "w") as f:
    f.write("tolerance 2.0\n".format(packing_tolerance))
    f.write("filetype pdb\n")
    f.write("output {0}\n".format(starting_pdb_file))
    f.write("\n")
    f.write("structure water.pdb\n")
    f.write("  number {0}\n".format(water))
    f.write("  inside cube 2. 2. 2. {0}\n".format(starting_box_size))
    f.write("end structure\n")
    f.write("\n")
    f.write("add_box_sides 1.0\n")


Now we run ```packmol``` itself, and make sure that everything worked ok:

In [ ]:
#%%bash
!packmol < packmol.in > packmol.out

In [ ]:
with open('packmol.out', 'r') as f:
    text = f.read()
    print(text)

Let's take a look at the simulation box that you've just generated: 

In [ ]:
with open('system.pdb', 'r') as f:
    text = f.read()
    print(text)

Notice here that the only particles in the simulation box so far are hydrogen and oxygen atoms. This is fine for the TIP3P water model, however a bit later we will have to adjust this file to include the "M" dummy atoms when we use the TIP4P water model. 

In [ ]:
#correct and load pdb file
# edit the pdb file to have the correct pbc:
pbcBoxLine = "CRYST1 {:<08} {:<08} {:<08}  90.00  90.00  90.00 P 1           1"\
        .format(starting_box_size,starting_box_size,starting_box_size)

# open the file to read:
foundCrystLine = False
with open(starting_pdb_file,'r+') as f:
    data = f.readlines()
    crystre = re.compile('^CRYST1')
    for idx,line in enumerate(data):
        if re.match(crystre,line):
            lineNum = idx
            foundCrystLine = True
            break

with open(starting_pdb_file, 'w+') as f:
    for idx,entry in enumerate(data):
        if idx == lineNum:
            f.write(pbcBoxLine + '\n')
        else:
            f.write(entry)
            
pdb = PDBFile(starting_pdb_file)

#### Defining the MD Simulation ####

We are now ready to setup the MD simulation objects using the ```OpenMM``` python module. The process for this is _exactly_ the same as that in the introductory workshop - we need to define: 

1. What forcefield we are going to use (this defines how bonded and non-bonded atoms interact with each other)
1. What the system topology is (i.e. which atoms are connected to which atoms etc.)
1. How we will control the simulation temperature and pressure (e.g. using a _thermostat_ or a _barostat_), and
1. How we will iterate Newton's equations of motion (i.e. the _integrator_)

Run the cells below. They will setup the simulation in the following way: 

1. We will apply the TIP3P water model forcefield and store it in  the ```tip3p_forcefield``` object. 
1. We combine the ```tip3p_forcefield``` object with the topology in our PDB file ```system.pdb``` (generated above using packmol) to define the MD simulation, which we store in the ```tip3p_system``` object. 
1. We add a barostat (the ```MonteCarloBarostat```) to the simulation using ```OpenMM```'s ```addForce``` module. 
1. We use ```OpenMM```'s ```NoseHooverIntegrator``` to iterate the equations of motion.


In [ ]:
tip3p_forcefield = ForceField('tip3p.xml')

In [ ]:
tip3p_system = tip3p_forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=14*angstrom,
    constraints=None,
    rigidWater=False)

In [ ]:
tip3p_system.addForce(MonteCarloBarostat(simPressure, simTemperature, 1))

In [ ]:
tip3p_integrator = NoseHooverIntegrator(
    simTemperature, 
    0.1/picosecond, 
    simTimestep
)

Finally, we tie all of these details together in the ```tip3p_simulation``` object, via ```OpenMM```'s ```Simulation``` module:

In [ ]:
tip3p_simulation = Simulation(pdb.topology, tip3p_system, tip3p_integrator)
tip3p_simulation.context.setPositions(pdb.positions)

Now we are ready to run!! 

First things first - we have to minimise the energy of our water box to a local minimum on the potential energy surface. Run the next cell to do this: 

In [ ]:
print("Running Energy Minimisation:")
t0 = time.time()
tip3p_simulation.minimizeEnergy()
t1 = time.time()
minTime = t1-t0
print("Minimisation took {} seconds".format(minTime))

The arrangement of waters in your box should now correspond to a local energy minimisation, meaning that you are good to go - run the MD simulation!

Before we do though, we want to make sure that we store the relevant data from the MD trajectory. The cell below will take care of this for you, and will track the progress of the MD simulation in a file ```tip3p_Equil.dcd```:

In [ ]:
tip3p_simulation.reporters.append(DCDReporter('tip3p_Equil.dcd', 10))
tip3p_simulation.reporters.append(StateDataReporter(
    stdout, 
    10, 
    step=True,
    temperature=True, 
    potentialEnergy=True, 
    kineticEnergy=True,
    totalEnergy=True, 
    volume=True, 
    density=True
))

Run the next cell to run your MD simulation!

In [ ]:
print('Start Simulation')
t0 = time.time()
tip3p_simulation.step(simNumSteps)
t1 = time.time()
simTime = t1-t0
print("Simulation took {} seconds for {} timesteps".format(simTime,simNumSteps)) 

### TIP4P-Ew Water Simulation ###

We'll now re-simulate the same box of water using a different force field for a comparison. The setup is a little simpler here, since we do not have to load python modules, construct the simulation box with ```packmol```, define the MD simulation parameters etc. 

We only need to setup the new MD simulation object using ```OpenMM```. The process for this is _exactly_ the same as that above - recall, we need to define: 

1. What forcefield we are going to use (this defines how bonded and non-bonded atoms interact with each other)
1. What the system topology is (i.e. which atoms are connected to which atoms etc.)
1. How we will control the simulation temperature and pressure (e.g. using a _thermostat_ or a _barostat_), and
1. How we will iterate Newton's equations of motion (i.e. the _integrator_)

Run the cells below. They will setup the simulation in the following way: 

1. We will apply the TIP4P-Ew water model forcefield and store it in  the ```tip4pew_forcefield``` object. 
1. We combine the ```tip4pew_forcefield``` object with the topology in our PDB file ```system.pdb``` (generated above using packmol) to define the MD simulation, which we store in the ```tip4pew_system``` object. 
1. We add a barostat (the ```MonteCarloBarostat```) to the simulation using ```OpenMM```'s ```addForce``` module. 
1. We use ```OpenMM```'s ```NoseHooverIntegrator``` to iterate the equations of motion.

Notice that we are only changing **1 aspect** of our MD simulation - the water model itself - _all other aspects of the MD simulation are the same._ This means that any & all differences between our TIP3P and TIP4P-Ew simulations can only be caused by the differences in the water models themselves. 

In [ ]:
tip4pew_forcefield = ForceField('tip4pew.xml')

In [ ]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.addExtraParticles(tip4pew_forcefield)
PDBFile.writeFile(modeller.topology, modeller.positions, open('system_tip4pew.pdb', 'w'))
tip4pew_pdb = PDBFile('system_tip4pew.pdb','M')

In [ ]:
tip4pew_system = tip4pew_forcefield.createSystem(tip4pew_pdb.topology,nonbondedMethod=PME,nonbondedCutoff=14*angstrom,constraints=None,rigidWater=False)
tip4pew_system.addForce(MonteCarloBarostat(simPressure, simTemperature, 1))
tip4pew_integrator = NoseHooverIntegrator(
    simTemperature, 
    0.1/picosecond, 
    simTimestep
)
tip4pew_simulation = Simulation(tip4pew_pdb.topology, tip4pew_system, tip4pew_integrator)
tip4pew_simulation.context.setPositions(tip4pew_pdb.positions)

You now have your TIP4P-Ew water MD simulation set up and ready to run. Now it's your turn to write some of your own code!

#### Exercise ####

1. Add a new cell to the Jupyter notebook below. Using the **energy minimisation** python code above from your TIP3P simulation as a template, minimise the energy of your water simulation box using the TIP4P water model. _NOTE - remember to change 'tip3p' to 'tip4pew' in all variable names!_
1. Add a new cell to the Jupyter notebook below. Using the **MD simulation** python code above from your TIP3P simulation as a template, run the MD simulation of your water simulation box using the TIP4P water model. _NOTE - remember to tell your Jupyter notebook to record your trajectory in a DCD file first!!_

## Analysis - Water Structure & the Radial Distribution Function ##

The two extreme scenarios of molecular interaction are solids, having molecules with strong interactions, which are aligned in a rigid crystal, and gases, having molecules with no to minimal interactions, with essentially no regular arrangement in their structure. The amount of interaction (and thus structure) of molecules in a liquid falls in the middle. This is one of the things that makes liquids so interesting - _and the main reason almost all chemistry occurs in a solvent!_ 

The structure in a liquid can be characterized by a radial distribution function (RDF), g(r). An RDF is a measure of the radial frequency of particles around some point in space. Typically, we use RDFs to measure the likelihood of finding an atom type (e.g. oxygen) around another atom type (e.g. hydrogen). For a monatomic gas, g(r) is flat, because any atom is a random distance from all other atoms. There is no preferred distance; all are equally likely. For a crystalline solid, g(r) has regular, sharp peaks, because atoms are regularly spaced from each other - you will be familiar with this from X-ray diffraction patterns. In a solid, only some distances are likely; others are essentially forbidden. 

Liquids, on the other hand, are structured locally but random at long range. This means a liquid g(r) resembles that of a solid at short-range, but that of a gas at long-range. Most liquids typically exhibit a few distinct short range peaks, these correspond to **solvation shells**. The position of these peaks indicate the length scale of each solvation shell, and the height of that peak indicates how pronounced the corresponding shell is in structure, relative to the surrounding bulk liquid environment. Eventually, as the nonlocal density becomes identical to the bulk density, g(r) flattens and tends to the bulk value of 1. 

For water, the amount of local structure can be characterized via the O-O, O-H and H-H g(r) functions (i.e. "oxygens around oxygens", "hydrogens around oxygens" and "hyrogens around hydrogens"), which are shown in the Figure below. This figure is produced via neutron diffraction measurements, however MD is also capable of producing RDFs. These RDFs show that: 

1. The hydrogen bonding structure in bulk water (i.e. the O-H g(r))  extends well beyond the first solvation shell (there are two hydrogen bond interactions at ~2 and 3.5 $\mathring A$, respectively). 
1. The interaction length between neighbouring water molecules (via the O-O g(r)). The figure below shows that water arranges itself into ~3 distinct solvation shells at ~2.5, 4.5 and 7 $\mathring A$, respectively. 

So, water is **really** structured!! Now that we understand this, we can begin to understand the physical and chemical properties of water, and why some of them change in the presence of solutes and interfaces (as we will see in the next workshop). 

<img src="https://static-02.hindawi.com/articles/isrn/volume-2013/279463/figures/279463.fig.0021.jpg" align="left" width="500" height="500"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/e/ea/Molecular_Schematic_for_Interpreting_a_Radial_Distribution_Function.png" align="center" width="400" height="400"/>


Let's take our MD trajectory and use the MDAnalysis python module to calculate the same RDFs as shown in the Figure above. 

In [ ]:
import MDAnalysis as mda
import MDAnalysis.analysis.rdf as rdf

MDAnalysis is extremely powerful, and relies on what is known as a 'universe' (essentially, all the information that defines an MD trajectory, i.e. the topology, the coordinates and the force field). Once we have defined a universe based on our trajectory, we can perform the RDF analysis. Let's store the universe in the variable ```tip3p_u```:

In [ ]:
tip3p_u = mda.Universe('system.pdb','tip3p_Equil.dcd')

We now need to tell MDAnalysis which pairs of atoms we want to investigate. To start with, let's look at the main O-O interaction in your bulk water MD simulation. We will use the ```select_atoms``` function to do this, and store the list of oxyegen atoms in the variable ```O```:

In [ ]:
O = tip3p_u.select_atoms('name O')

And then tell MDAnalysis the details of the RDF that we are after: 

In [ ]:
rdf_calc = rdf.InterRDF(O,O,nbins=100,range=(1.0,10))

Here, we are using MDAnalysis's ```InterRDF``` function to calculate the RDF between each oxygen and every other oxygen. We measure the RDF between 1 $\mathring A$ and 10 $\mathring A$ (see the ```range``` variable) at 100 points (see the ```nbins``` variable).

Now let's run the analysis -  the result will be stored in the ```rdf_calc``` object:

In [ ]:
rdf_calc.run()

This should only take a few seconds. When it is finished, we can plot the results and look at the RDF! We will use the ```matplotlib``` python module, as we have before, to do this. Note that the horizontal and vertical axis data we want to plot are actually now in the ```rdf_calc``` object, as the ```rdf_calc.bins``` and ```rdf_calc.rdf``` variables, respectively. So we will pass these two variables to the plot command below:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(rdf_calc.bins,rdf_calc.rdf,label="TIP3P")
plt.xlabel("r ($\mathrm{\AA}$)")
plt.ylabel("g(r)")
plt.ylim([0.0, 15])

#### Exercise ####

1. Use the code template below to repeat the RDF analysis for the ??, ?? and ?? water model MD simulations that you ran above. 

## Analysis - Self Diffusion Coefficients in Bulk Water ##